In [1]:
import numpy as np
import scipy.sparse
import sklearn.utils.extmath

In [2]:
np.random.seed(0)
A = np.random.normal(size = (2000, 2000))

In [20]:
def low_rank_approx(A, svd_func, rank = 1):
    if svd_func == np.linalg.svd:
        U_, S_, VT_ = svd_func(A, full_matrices = False)
        U = U_[:, :rank]
        S = S_[ :rank]
        VT = VT_[ :rank, :]
    elif svd_func == scipy.sparse.linalg.svds:
        U, S, VT = svd_func(A, rank)
    elif svd_func == sklearn.utils.extmath.randomized_svd:
        U, S, VT = svd_func(A, rank, random_state = 0)

    return U * S @ VT # same as U @ np.diag(S) @ VT

Измерим время:

In [21]:
%%timeit
low_rank_approx(A, np.linalg.svd)

3.73 s ± 374 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
low_rank_approx(A, scipy.sparse.linalg.svds)

484 ms ± 91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
low_rank_approx(A, sklearn.utils.extmath.randomized_svd)

96.7 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Время отличается на порядок.

Измерим нормы отклонений:

In [4]:
A_svd = low_rank_approx(A, np.linalg.svd)
np.linalg.norm(A - A_svd, ord = 'fro')

1996.8937254640764

In [9]:
A_svds = low_rank_approx(A, scipy.sparse.linalg.svds)
np.linalg.norm(A - A_svds, ord = 'fro')

1996.8937254640762

In [10]:
A_rsvd = low_rank_approx(A, sklearn.utils.extmath.randomized_svd)
np.linalg.norm(A - A_rsvd, ord = 'fro')

1996.9956974865386

Нормы отклонений почти одинаковые. Сами матрицы тоже почти одинаковые:

In [71]:
np.linalg.norm(A_rsvd - A_svds, ord = 'fro')
# в расчёте на 2000**2 элементов это немного

123.44299771021934